In [ ]:
from services.db_service import DbService

In [ ]:
decks_collection = DbService().deck_collection()
personal_pool_collection = DbService().personal_pool_collection()

In [ ]:
pipeline = [
    {"$project": {"all_cards": {"$concatArrays": [
        "$maindeck", "$sideboard"]}, "name": 1}},
    {"$unwind": "$all_cards"},
    {"$group": {"_id": {"_id": "$_id", "card_name": "$all_cards",
                        "name": "$name"}, "count": {"$sum": 1}}},
    {"$group": {"_id": "$_id._id", "name": {"$first": "$_id.name"},
                "cards": {"$push": {"name": "$_id.card_name", "count": "$count"}}}}
]

decks = list(decks_collection.aggregate(pipeline))
decks

In [ ]:
personal_pool = {}
for card in personal_pool_collection.find({}, {"name": 1, "count": 1, "_id": 0}):
    personal_pool[card["name"]] = card["count"]

personal_pool

In [ ]:
usable_decks = []
for deck in decks:
    can_build = True
    for card in deck["cards"]:
        if card["name"] not in personal_pool or personal_pool[card["name"]] < card["count"]:
            can_build = False
            break
    if can_build:
        usable_decks.append(deck["_id"])

In [ ]:
for deck_id in usable_decks:
    deck = decks_collection.find_one({"_id": deck_id})
    print(f"{deck["name"]} ({deck["_id"]})")